# Nanoporous Materials Screening Experiments

This notebook demonstrates the use of ST4SD REST-API to launch adsorption experiments directly from the Jupyter Notebook.

In [ ]:
from __future__ import print_function
import experiment.service.db
import logging
import os
import pandas as pd
import json

logging.basicConfig(format='%(levelname)-9s %(name)-15s: %(funcName)-20s %(asctime)-15s: %(message)s')
root=logging.getLogger()
root.setLevel(logging.WARNING)

## Connect to target OpenShift cluster

In [ ]:
 # Fill this with the https:// end-point of your ST4SD registry
 # OR set the URL via the ST4SD_URL environment variable before running this notebook
st4sd_url = os.getenv('ST4SD_URL', None)

if st4sd_url is None:
  raise ValueError("You need to fill in ST4SD_URL above.")

### Authenticate to workflow scheduler

1. Visit URL printed below
1. After logining in you will be presented with an authentication token that you will provide to the experiment.service.db.ExperimentRestAPI wrapper in a python cell below.

In [ ]:
auth_url = '/'.join((st4sd_url, 'authorisation/token'))
if os.getenv('ST4SD_BEARER_KEY') is None:
    print("Authorisation Token URL:", auth_url)
else:
    print("Will use Bearer Key from `ST4SD_BEARER_KEY` environment variable instead of an AuthorizationToken")

### Connect to workflow scheduler API

In [ ]:
# Unless you have already populated the `ST4SD_BEARER_KEY` environment variable with a suitable Bearer KEY, 
# set `token` to what you copied above 
token = None

try:
    api = experiment.service.db.ExperimentRestAPI(st4sd_url,
                                                  max_retries=2,
                                                  secs_between_retries=1,
                                                  cc_bearer_key=os.getenv('ST4SD_BEARER_KEY'),
                                                  cc_auth_token=token
    )
except experiment.service.errors.UnauthorisedRequest as e:
    print(f"Visit {auth_url} to retrieve your authentication token. Then use it to set the value of "
          "\"auth_token\" in the above cell. Execute that cell and then execute this one.")
else:
    print(f"You've successfully authenticated to {st4sd_url}")

## Define a subset of materials to be screened

In [ ]:
bundled = ["CoRE2019/GUJVOX_clean", "CoRE2019/VAZLUF_clean", "CoRE2019/WEMFEB_clean"]
cif_files_dat='\n'.join(bundled)
print(cif_files_dat)

## Define screening protocol

In [ ]:
experimentId = "nanopore-adsorption-experiment"
definition = {
  "base": {
    "packages": [{
      "name": "nanopore-adsorption-experiment",
      "source": {
        "git": {
          "location": {
            "url": "https://github.com/st4sd/nanopore-adsorption-experiment.git",
            "commit": "4cf04f00048c5b129c77117940590be866816f18"
          }
        }
      },
      "config": {
        "path": "conf/flowir_package.yaml",
        "manifestPath": "manifest.yaml"
      }
    }]
  },
  "metadata": {
    "package": {
      "name": "nanopore-adsorption-experiment",
      "tags": ["latest", "1.0.0"],
      "maintainer": "rneumann@br.ibm.com",
      "license": "Apache 2.0",
      "description": "Uses the RASPA software package to simulate adsorption isotherms on nanoporous materials.",
      "keywords": [
        "openshift"
      ]
    }
  },
  "parameterisation": {
    "presets": {
      "runtime": {
        "args": [
          "--registerWorkflow=yes"
        ]
      },
      "platform": "openshift"
    },
    "executionOptions": {
      "variables": [
        {"name": "numberOfNanopores"},
        {"name": "raspa_memory"},
        {"name": "externalTemperature_K"},
        {"name": "externalPressure_Pa"},
        {"name": "gasComposition"}
      ],
    }
  }
}
api.api_experiment_push(definition)

## Launch experiment

In [ ]:
# Trim any empty new lines - we use the number of lines to determine the number of replicas
cif_files_dat = cif_files_dat.rstrip()

experimentConfiguration = {
# Uncomment the "volumes" field below if you want to use a PVC instead of the bundled cif files.
# It will also be necessary to comment the "nanopore-database: ../cif:copy" line in manifest.yaml
#     "volumes": [{
#         "type": {"persistentVolumeClaim": "nanopore-database-pvc"},
#         "applicationDependency": "nanopore-database"
#     }],
    "additionalOptions": [
        "--useMemoization=true",
        "--registerWorkflow=true"
    ],
    "orchestrator_resources": {
      "cpu": "1.0",
      "memory": f'{int(1 + len(cif_files_dat.splitlines()) / 100)}Gi'
    },
   "inputs": [{
       "filename": "cif_files.dat",
       "content": cif_files_dat
   }],
   "variables": {
       "numberOfNanopores": len(cif_files_dat.splitlines()),
       "raspa_memory": "2Gi",
       "externalTemperature_K": 298,
       "externalPressure_Pa": '1000,2000,5000,10000,20000,50000,100000',
       "gasComposition": '{\\"CO2\\":1.0}'
   }
}

rest_uid = api.api_experiment_start(experimentId, experimentConfiguration)

In [ ]:
# It's not recommended to use include_properties=['*'] while the experiment is still running
# status = api.api_rest_uid_status(rest_uid, include_properties=['*'])
status = api.api_rest_uid_status(rest_uid)

print(json.dumps(status, indent=2))

## Getting the Inputs

In [ ]:
print(status['experiment']['metadata']['registry']['interface'])

## Getting the Properties

In [ ]:
pd.set_option('display.max_colwidth', None)
properties_dict = api.api_request_get(f'instances/{rest_uid}/properties')
df = pd.DataFrame.from_dict(properties_dict)
df